In [13]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import VarianceThreshold
from sklearn.metrics import classification_report
from sklearn.metrics import make_scorer, f1_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [14]:
adult_18 = pd.read_csv('adult_18.csv')

In [15]:
# Get the list of columns
columns_list = adult_18.columns.tolist()
columns_list

['FPX',
 'FMX',
 'HHX',
 'INTV_QRT',
 'WTIA_SA',
 'WTFA_SA',
 'SEX',
 'HISPAN_I',
 'R_MARITL',
 'MRACRPI2',
 'RACERPI2',
 'MRACBPI2',
 'AGE_P',
 'RECTYPE',
 'SRVY_YR',
 'INTV_MON',
 'REGION',
 'PSTRAT',
 'PPSU',
 'PROX1',
 'PROX2',
 'LATEINTA',
 'PROXYSA',
 'PAR_STAT',
 'DOINGLWA',
 'WHYNOWKA',
 'EVERWRK',
 'SUPERVIS',
 'WRKCATA',
 'BUSINC1A',
 'LOCALL1B',
 'WRKLONGH',
 'HOURPDA',
 'PDSICKA',
 'ONEJOB',
 'WRKLYR4',
 'INDSTRN1',
 'INDSTRN2',
 'OCCUPN1',
 'OCCUPN2',
 'YRSWRKPA',
 'DIFAGE2',
 'HYPEV',
 'HYPDIFV',
 'HYPMDEV2',
 'HYPMED2',
 'CHLEV',
 'CHLYR',
 'CHLMDEV2',
 'CHLMDNW2',
 'CHDEV',
 'ANGEV',
 'MIEV',
 'HRTEV',
 'STREV',
 'EPHEV',
 'COPDEV',
 'ASPMEDEV',
 'ASPMEDAD',
 'ASPMDMED',
 'ASPONOWN',
 'AASMEV',
 'AASSTILL',
 'AASAGE',
 'AASAGE16',
 'AASMYR',
 'AASERYR1',
 'AASMHSP',
 'AASMPMED',
 'AASMTYP',
 'AASMCAN',
 'AASMDTP2',
 'AASWMP',
 'AASCLASS',
 'AAS_REC',
 'AAS_RES',
 'AAS_MON',
 'AAPENVLN',
 'AASYMPT',
 'AARESCUE',
 'AAACTLIM',
 'AASSTAT',
 'AASEMP',
 'AASWKREL',
 'AASDWKRL

In [16]:
prostate_cancer_data18 = adult_18[adult_18.CNKIND20.notna()]
#prostate_cancer_data18 = pd.read_csv(prostate_cancer_data18.csv)
prostate_cancer_data18.shape

(1167, 742)

### SKNMCAN ISN'T IN THE GENERATED DATAFRAME?

In [17]:
prostate_cancer_data18.to_csv('prostate_cancer_data18.csv', index=False)

#Renaming the Prostate Cancer column to PROSTCAN_A
prostate_cancer_data18.rename(columns={'CNKIND20':'PROSTCAN_A','AGE_P':'AGEP_A','CNKIND22':'SKNNMCAN_A',
                                      'CNKIND16':'SKNMCAN_A', 'CNKIND31':'OTHERCANP_A','CNKIND23':'SKNDCAN_A', 
                                       'CNKIND16':'MELANCAN_A','AWEIGHTP':'WEIGHTLBTC_A','CNKIND14':'LUNGCAN_A',
                                       'CNKIND15':'LYMPHCAN_A','CNKIND7':'COLONCAN_A','AHEIGHT':'HEIGHTTC_A',
                                       'CNKIND1':'BLADDCAN_A','CNKIND19':'PANCRCAN_A','CNKIND27':'THROACAN_A',
                                       'CNKIND8':'ESOPHCAN_A','CNKIND21':'RECTUCAN_A','CNKIND9':'GALLBCAN_A',
                                       'CNKIND3':'BONECAN_A','CNKIND12':'LEUKECAN_A','CNKIND17':'MOUTHCAN_A',
                                       'CNKIND13':'LIVERCAN_A','DEP_1':'DEPFREQ_A','CNKIND11':'LARYNCAN_A',
                                       'PAIN_2A':'PAIFRQ3M_A','CNKIND5':'BREASCAN_A','ALUNIT17':'ANXFREQ_A',
                                       'RACERPI2':'RACEALLP','HISPAN_I':'HOUYRSLIV_A','SMKLSTB1':'SMOKELSEV_A'}, inplace=True)


C:\Users\HP 10th Gen\AppData\Local\Temp\ipykernel_37668\3729412839.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prostate_cancer_data18.rename(columns={'CNKIND20':'PROSTCAN_A','AGE_P':'AGEP_A','CNKIND22':'SKNNMCAN_A',


In [18]:

columns = prostate_cancer_data18.columns.sort_values()

# Correct variable name: S_columns
S_columns = [i for i in columns if i.startswith("SKN")]

# Use the correct case for the variable
prostate_cancer_data18[S_columns].notnull().sum()


SKNDCAN_A     1167
SKNNMCAN_A    1167
dtype: int64

In [19]:
prostate_cancer_data18.sample(5)

,FPX,FMX,HHX,INTV_QRT,WTIA_SA,WTFA_SA,SEX,HOUYRSLIV_A,R_MARITL,MRACRPI2,RACEALLP,MRACBPI2,AGEP_A,RECTYPE,SRVY_YR,INTV_MON,REGION,PSTRAT,PPSU,PROX1,PROX2,LATEINTA,PROXYSA,PAR_STAT,DOINGLWA,WHYNOWKA,EVERWRK,SUPERVIS,WRKCATA,BUSINC1A,LOCALL1B,WRKLONGH,HOURPDA,PDSICKA,ONEJOB,WRKLYR4,INDSTRN1,INDSTRN2,OCCUPN1,OCCUPN2,YRSWRKPA,DIFAGE2,HYPEV,HYPDIFV,HYPMDEV2,HYPMED2,CHLEV,CHLYR,CHLMDEV2,CHLMDNW2,CHDEV,ANGEV,MIEV,HRTEV,STREV,EPHEV,COPDEV,ASPMEDEV,ASPMEDAD,ASPMDMED,ASPONOWN,AASMEV,AASSTILL,AASAGE,AASAGE16,AASMYR,AASERYR1,AASMHSP,AASMPMED,AASMTYP,AASMCAN,AASMDTP2,AASWMP,AASCLASS,AAS_REC,AAS_RES,AAS_MON,AAPENVLN,AASYMPT,AARESCUE,AAACTLIM,AASSTAT,AASEMP,AASWKREL,AASDWKRL,ULCEV,ULCYR,CANEV,BLADDCAN_A,CNKIND2,BONECAN_A,CNKIND4,BREASCAN_A,CNKIND6,COLONCAN_A,ESOPHCAN_A,GALLBCAN_A,CNKIND10,LARYNCAN_A,LEUKECAN_A,LIVERCAN_A,LUNGCAN_A,LYMPHCAN_A,MELANCAN_A,MOUTHCAN_A,CNKIND18,PANCRCAN_A,PROSTCAN_A,RECTUCAN_A,SKNNMCAN_A,SKNDCAN_A,CNKIND24,CNKIND25,CNKIND26,THROACAN_A,CNKIND28,CNKIND29,CNKIND30,OTHERCANP_A,DIBEV1,DIBPRE2,DIBPILL1,INSLN1,AHAYFYR,SINYR,CBRCHYR,KIDWKYR,LIVYR,JNTSYMP,JMTHP1,JMTHP2,JMTHP3,JMTHP4,JMTHP5,JMTHP6,JMTHP7,JMTHP8,JMTHP9,JMTHP10,JMTHP11,JMTHP12,JMTHP13,JMTHP14,JMTHP15,JMTHP16,JMTHP17,JNTCHR,JNTHP,ARTH1,ARTHLMT,PAINECK,PAINLB,PAINLEG,PAINFACE,AMIGR,ACOLD2W,AINTIL2W,PREGNOW,PREGFLYR,HRAIDNOW,HRAIDEV,AHEARST1,AVISION,ABLIND,LUPPRT,HYPYR1,AWZMSWKP,AROUTINP,CANAGE1,CANAGE2,CANAGE3,CANAGE4,CANAGE5,CANAGE6,CANAGE7,CANAGE8,CANAGE9,CANAGE10,CANAGE11,CANAGE12,CANAGE13,CANAGE14,CANAGE15,CANAGE16,CANAGE17,CANAGE18,CANAGE19,CANAGE20,CANAGE21,CANAGE22,CANAGE23,CANAGE24,CANAGE25,CANAGE26,CANAGE27,CANAGE28,CANAGE29,CANAGE30,DIBAGE1,ACHRC14A,ADURA14A,ADURB14A,AFLHC19_,AFLHC20_,AFLHC21_,AFLHC22_,AFLHC23_,AFLHC24_,AFLHC25_,AFLHC26_,AFLHC27_,AFLHC28_,AFLHC29_,AFLHC30_,AFLHC31_,AFLHC32_,AFLHC33_,AFLHC34_,AFLHCA1,AFLHCA10,AFLHCA11,AFLHCA12,AFLHCA13,AFLHCA15,AFLHCA16,AFLHCA17,AFLHCA18,AFLHCA2,AFLHCA3,AFLHCA4,AFLHCA5,AFLHCA6,AFLHCA7,AFLHCA8,AFLHCA9,AFLHCA90,AFLHCA91,AHSTATYR,ALCHRC1,ALCHRC10,ALCHRC11,ALCHRC12,ALCHRC13,ALCHRC15,ALCHRC16,ALCHRC17,ALCHRC18,ALCHRC19,ALCHRC2,ALCHRC20,ALCHRC21,ALCHRC22,ALCHRC23,ALCHRC24,ALCHRC25,ALCHRC26,ALCHRC27,ALCHRC28,ALCHRC29,ALCHRC3,ALCHRC30,ALCHRC31,ALCHRC32,ALCHRC33,ALCHRC34,ALCHRC4,ALCHRC5,ALCHRC6,ALCHRC7,ALCHRC8,ALCHRC9,ALCHRC90,ALCHRC91,ALCHRONR,ALCNDRT,ALDURA1,ALDURA10,ALDURA11,ALDURA12,ALDURA13,ALDURA15,ALDURA16,ALDURA17,ALDURA18,ALDURA19,ALDURA2,ALDURA20,ALDURA21,ALDURA22,ALDURA23,ALDURA24,ALDURA25,ALDURA26,ALDURA27,ALDURA28,ALDURA29,ALDURA3,ALDURA30,ALDURA31,ALDURA32,ALDURA33,ALDURA34,ALDURA4,ALDURA5,ALDURA6,ALDURA7,ALDURA8,ALDURA9,ALDURA90,ALDURA91,ALDURB1,ALDURB10,ALDURB11,ALDURB12,ALDURB13,ALDURB15,ALDURB16,ALDURB17,ALDURB18,ALDURB19,ALDURB2,ALDURB20,ALDURB21,ALDURB22,ALDURB23,ALDURB24,ALDURB25,ALDURB26,ALDURB27,ALDURB28,ALDURB29,ALDURB3,ALDURB30,ALDURB31,ALDURB32,ALDURB33,ALDURB34,ALDURB4,ALDURB5,ALDURB6,ALDURB7,ALDURB8,ALDURB9,ALDURB90,ALDURB91,ALHCA14A,ALTIME1,ALTIME10,ALTIME11,ALTIME12,ALTIME13,ALTIME15,ALTIME16,ALTIME17,ALTIME18,ALTIME19,ALTIME2,ALTIME20,ALTIME21,ALTIME22,ALTIME23,ALTIME24,ALTIME25,ALTIME26,ALTIME27,ALTIME28,ALTIME29,ALTIME3,ALTIME30,ALTIME31,ALTIME32,ALTIME33,ALTIME34,ALTIME4,ALTIME5,ALTIME6,ALTIME7,ALTIME8,ALTIME9,ALTIME90,ALTIME91,ALUNIT1,ALUNIT10,ALUNIT11,ALUNIT12,ALUNIT13,ALUNIT15,ALUNIT16,ANXFREQ_A,ALUNIT18,ALUNIT19,ALUNIT2,ALUNIT20,ALUNIT21,ALUNIT22,ALUNIT23,ALUNIT24,ALUNIT25,ALUNIT26,ALUNIT27,ALUNIT28,ALUNIT29,ALUNIT3,ALUNIT30,ALUNIT31,ALUNIT32,ALUNIT33,ALUNIT34,ALUNIT4,ALUNIT5,ALUNIT6,ALUNIT7,ALUNIT8,ALUNIT9,ALUNIT90,ALUNIT91,ATIME14A,AUNIT14A,BEDDAYR,FLA1AR,FLCARRY,FLCLIMB,FLGRASP,FLPUSH,FLREACH,FLRELAX,FLSHOP,FLSIT,FLSOCL,FLSTAND,FLSTOOP,FLWALK,SPECEQ,WKDAYR,ALC12MNO,ALC12MTP,ALC1YR,ALC5UPN1,ALC5UPT1,ALC5UPY1,ALCAMT,ALCLIFE,BINGE1,CIG30D2,CIGAREV2,CIGCUR2,CIGDAMO,CIGQTYR,CIGSDA1,CIGSDA2,ECIG30D2,ECIGCUR2,ECIGEV2,MODLNGNO,MODLNGTP,MODNO,MODTP,PIPECUR2,PIPEV2,SMKEV,SMKLSCR2,SMOKELSEV_A,SMKNOW,SMKQTNO,SMKQTTP,SMKREG,STRNGNO,STRNGTP,VIGLNGNO,VIGLNGTP,VIGNO,VIGTP,SMKSTAT2,SMKQTY

In [8]:
# Dropping of columns not needed


required_columns18 = ['AGEP_A','SKNNMCAN_A','OTHERCANP_A','SKNDCAN_A','MELANCAN_A','WEIGHTLBTC_A','LUNGCAN_A',
                     'LYMPHCAN_A','COLONCAN_A','HEIGHTTC_A','BLADDCAN_A','PANCRCAN_A','THROACAN_A','ESOPHCAN_A',
                      'GALLBCAN_A','BONECAN_A','LEUKECAN_A','MOUTHCAN_A','LIVERCAN_A','DEPFREQ_A','LARYNCAN_A',
                      'BREASCAN_A','REGION','ANXFREQ_A','RACEALLP','HOUYRSLIV_A','SMOKELSEV_A','PROSTCAN_A']

#  Keep only the specified columns
columns_filtered = prostate_cancer_data18[required_columns18]

# Drop any unnamed columns
# Unnamed columns typically have names like 'Unnamed: 0'
unwanted_columns = [col for col in columns_filtered.columns if 'Unnamed:' in str(col)]
required_data18 = columns_filtered.drop(unwanted_columns, axis=1)

In [9]:
#missing_pct = prostate_cancer_data18.isna().mean()
#low_missingness_columns = missing_pct[missing_pct == 0].index
#cancer_reduced_missing = prostate_cancer_data18[low_missingness_columns]
#cancer_reduced_missing.shape

In [10]:
#unique_values = prostate_cancer_data18['PROSTCAN_A'].unique()
#print(unique_values)

In [11]:
from sklearn.feature_selection import VarianceThreshold
import pandas as pd

# Assuming `features` is your DataFrame containing the features

# Create a VarianceThreshold instance and fit to the features
#selector = VarianceThreshold()
#selector.fit(features)

# Create a DataFrame with feature names and their variances
#variances = pd.DataFrame({"features": features.columns, "variance": selector.variances_})

# Sort the DataFrame in descending order by variance
#variances = variances.sort_values(by="variance", ascending = False)

# Display the variances DataFrame
#print(variances)


In [12]:
adult_19 = pd.read_csv('adult_19.csv')
prostate_cancer_data19.to_csv('prostate_cancer_data19.csv', index=False)
prostate_cancer_data18.to_csv('prostate_cancer_data18.csv', index=False)

NameError: name 'prostate_cancer_data19' is not defined

In [ ]:
prostate_cancer_data19 = adult_19[adult_19.PROSTCAN_A.notna()]
prostate_cancer_data19.shape

In [ ]:
#REPLACING OF COLUMN NAME
prostate_cancer_data19.rename(columns={'EDUC_A':'EDUCP_A', 'RACEALLP_A':'RACEALLP', 'SKNDKCAN_A':'SKNDCAN_A' }, inplace= True)

In [ ]:
prostate_cancer_data19.sample(10)

In [ ]:
#SKNDCAN_A AND RACEALLP AREN'T IN SKIN CANCER DATA

In [ ]:
# Dropping of columns not needed


required_columns19 = ['AGEP_A','SKNNMCAN_A','OTHERCANP_A','MELANCAN_A','WEIGHTLBTC_A','RACEALLP','LUNGCAN_A',
                     'LYMPHCAN_A','COLONCAN_A','HEIGHTTC_A','BLADDCAN_A','PANCRCAN_A','THROACAN_A','ESOPHCAN_A',
                      'GALLBCAN_A','BONECAN_A','LEUKECAN_A','MOUTHCAN_A','LIVERCAN_A','DEPFREQ_A','LARYNCAN_A',
                      'BREASCAN_A','REGION','ANXFREQ_A','HOUYRSLIV_A','SMOKELSEV_A','SKNDCAN_A','PROSTCAN_A']

#  Keep only the specified columns
columns_filtered = prostate_cancer_data19[required_columns18]

# Drop any unnamed columns
# Unnamed columns typically have names like 'Unnamed: 0'
unwanted_columns = [col for col in columns_filtered.columns if 'Unnamed:' in str(col)]
required_data19 = columns_filtered.drop(unwanted_columns, axis=1)

In [ ]:
adult_20 = pd.read_csv('adult_20.csv')
prostate_cancer_data20.to_csv('prostate_cancer_data20csv', index=False)

In [ ]:
prostate_cancer_data20 = adult_20[adult_20.PROSTCAN_A.notna()]
prostate_cancer_data20.shape

In [ ]:
# Dropping of columns not needed


required_columns20 = ['AGEP_A','SKNNMCAN_A','OTHERCANP_A','MELANCAN_A','WEIGHTLBTC_A','RACEALLP','LUNGCAN_A',
                     'LYMPHCAN_A','COLONCAN_A','HEIGHTTC_A','BLADDCAN_A','PANCRCAN_A','THROACAN_A','ESOPHCAN_A',
                      'GALLBCAN_A','BONECAN_A','LEUKECAN_A','MOUTHCAN_A','LIVERCAN_A','DEPFREQ_A','LARYNCAN_A',
                      'BREASCAN_A','REGION','ANXFREQ_A','HOUYRSLIV_A','SMOKELSEV_A','SKNDCAN_A','PROSTCAN_A']

#  Keep only the specified columns
columns_filtered = prostate_cancer_data19[required_columns20]

# Drop any unnamed columns
# Unnamed columns typically have names like 'Unnamed: 0'
unwanted_columns = [col for col in columns_filtered.columns if 'Unnamed:' in str(col)]
required_data20 = columns_filtered.drop(unwanted_columns, axis=1)

In [ ]:
adult_21 = pd.read_csv('adult_21.csv')
prostate_cancer_data21.to_csv('prostate_cancer_data21.csv', index=False)

In [ ]:
prostate_cancer_data21 = adult_21[adult_21.PROSTCAN_A.notna()]
prostate_cancer_data21.shape

In [ ]:
#REPLACING OF COLUMN NAME
prostate_cancer_data21.rename(columns={'RACEALLP_A':'RACEALLP', 'SKNDKCAN_A':'SKNDCAN_A' }, inplace= True)

In [ ]:
# Dropping of columns not needed


required_columns21 = ['AGEP_A','SKNNMCAN_A','OTHERCANP_A','MELANCAN_A','WEIGHTLBTC_A','RACEALLP','LUNGCAN_A',
                     'LYMPHCAN_A','COLONCAN_A','HEIGHTTC_A','BLADDCAN_A','PANCRCAN_A','THROACAN_A','ESOPHCAN_A',
                      'GALLBCAN_A','BONECAN_A','LEUKECAN_A','MOUTHCAN_A','LIVERCAN_A','DEPFREQ_A','LARYNCAN_A',
                      'BREASCAN_A','REGION','ANXFREQ_A','HOUYRSLIV_A','SMOKELSEV_A','SKNDCAN_A','PROSTCAN_A']

#  Keep only the specified columns
columns_filtered = prostate_cancer_data21[required_columns21]

# Drop any unnamed columns
# Unnamed columns typically have names like 'Unnamed: 0'
unwanted_columns = [col for col in columns_filtered.columns if 'Unnamed:' in str(col)]
required_data21 = columns_filtered.drop(unwanted_columns, axis=1)

In [ ]:
adult_22 = pd.read_csv('adult_22.csv')
prostate_cancer_data22.to_csv('prostate_cancer_data22.csv', index=False)

In [ ]:
prostate_cancer_data22 = adult_22[adult_22.PROSTCAN_A.notna()]
prostate_cancer_data22.shape

In [ ]:
#REPLACING OF COLUMN NAME
prostate_cancer_data22.rename(columns={'RACEALLP_A':'RACEALLP', 'SKNDKCAN_A':'SKNDCAN_A'}, inplace= True)

In [ ]:
# Dropping of columns not needed


required_columns22 = ['AGEP_A','SKNNMCAN_A','OTHERCANP_A','MELANCAN_A','WEIGHTLBTC_A','RACEALLP','LUNGCAN_A',
                     'LYMPHCAN_A','COLONCAN_A','HEIGHTTC_A','BLADDCAN_A','PANCRCAN_A','THROACAN_A','ESOPHCAN_A',
                      'GALLBCAN_A','BONECAN_A','LEUKECAN_A','MOUTHCAN_A','LIVERCAN_A','DEPFREQ_A','LARYNCAN_A',
                      'BREASCAN_A','REGION','ANXFREQ_A','HOUYRSLIV_A','SMOKELSEV_A','SKNDCAN_A','PROSTCAN_A']

#  Keep only the specified columns
columns_filtered = prostate_cancer_data22[required_columns22]

# Drop any unnamed columns
# Unnamed columns typically have names like 'Unnamed: 0'
unwanted_columns = [col for col in columns_filtered.columns if 'Unnamed:' in str(col)]
required_data22 = columns_filtered.drop(unwanted_columns, axis=1)

## Merging the csv files to create another Dataframe

In [ ]:
# Use this if all DataFrames now have the same columns
final_prostate_data = pd.concat([required_data18,required_data19,required_data20,required_data21,required_data22], ignore_index=True)


In [ ]:
final_prostate_data.sample(5)

In [ ]:
final_prostate_data.shape

In [ ]:
missing_pct = final_prostate_data.isna().mean()
low_missingness_columns = missing_pct[missing_pct == 0].index
cancer_reduced_missing = final_prostate_data[low_missingness_columns]
cancer_reduced_missing.shape

In [ ]:
# Calculate the percentage of missing values for each column.
missing_percentage = final_prostate_data.isna().mean()
missing_pct = 0.2

# Filter columns with less than 30% missing values.
columns_with_less_than_30_percent_missing = missing_percentage[missing_percentage < missing_pct].index
len_missing = len(columns_with_less_than_30_percent_missing)
print(f"number of columns with less than {missing_pct * 100}% missing: {len_missing} \n")
# List the variable names with less than 30% missing values.
for column_name in columns_with_less_than_30_percent_missing:
    print(column_name)